(thruster_allocation)=
## Thruster allocation

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
plt.style.use('paper')
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('pdf')

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from d2e2f.pipelines.trip_statistics import clean_statistics
from scipy.stats import norm
from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp
from d2e2f.visualization.visualize import plot_map, plot_trips
import statsmodels.api as sm
import geopandas as gp
from d2e2f.reporting import pop_index, reload_kedro
import yaml
catalog = reload_kedro()
from d2e2f.analyze import arange_trip_matrix

In [ ]:
with open('metadata.yml',mode='r') as file:
    metadata = yaml.safe_load(file)
ship=metadata['ship']
shipname = metadata['shipname']

df = catalog.load(f'{ship}.data_with_trip_columns')
df['time'] = df.index
loader_raw = catalog.load(f'{ship}.raw_data')
df_raw = loader_raw[list(loader_raw.keys())[-1]]()
df_raw.index = pd.to_datetime(df_raw['Timestamp'])
dt = pd.to_datetime(df_raw['Timestamp']).diff().mean()
t_start = df.iloc[0].name
t_end = df.iloc[-1].name

trip_statistics = catalog.load(f'{ship}.trip_statistics')
trip_statistics_clean = catalog.load(f'{ship}.trip_statistics_clean')
trip_statistics_clean['trip_no'] = trip_statistics_clean.index
trip_statistics_clean.index = pd.to_datetime(trip_statistics_clean['start_time'])
trip_statistics_clean.index.name = 'time'

harbours = catalog.load(f'params:{ship}.harbours')
min_time = catalog.load(f'params:{ship}.min_time')
max_time = catalog.load(f'params:{ship}.max_time')
min_distance = catalog.load(f'params:{ship}.min_distance')
max_distance = catalog.load(f'params:{ship}.max_distance')
min_start_speed = catalog.load(f'params:{ship}.min_start_speed')
max_time_diff = catalog.load(f"params:{ship}.max_time_diff")

In [ ]:
glue('harbour1', list(harbours.keys())[0])
glue('harbour2', list(harbours.keys())[1])
glue('min_time', min_time)
glue('max_time', max_time)
glue('min_distance', min_distance)
glue('max_distance', max_distance)
glue('min_start_speed', min_start_speed)
glue('shipname', shipname)
glue('max_time_diff', max_time_diff)
glue('dt',dt)
glue('t_start',str(t_start.date()))
glue('t_end',str(t_end.date()))
glue('no_samples',np.round(len(df)/(10**6), decimals=1))
glue('direction1',trip_statistics_clean['trip_direction'].unique()[0])
glue('direction2',trip_statistics_clean['trip_direction'].unique()[1])
glue('no_trips',len(trip_statistics_clean))

High correlation between the aft thruster consumption ratio $r_{aft}$ ([eq](eq:aft_thrust_ratio)) and fuel consumption 
was observed as shown in {numref}`fig:thruster_correlation`.

```{math}
:label: eq:aft_thrust_ratio
r_{aft} = \frac{C_{aft}}{C_{aft} + C_{fwd}}
```

$$r_{aft} = \frac{C_{aft}}{C_{aft} + C_{fwd}}$$

where $C_{aft}$ and $C_{fwd}$ are the fuel consumption origin from the aft and forward thruster.

```{glue:figure} fig:thruster_correlation
:name: "fig:thruster_correlation"

Correlation between the fuel consumption and the aft thruster ratio
```

This means that the fuel consumption is often higher when the thrust allocation is 50-50% between the aft and forward thruster, compared to when more of the thrust comes from the aft thruster.

In [ ]:
fig,ax=plt.subplots()

sns.regplot(data=trip_statistics_clean, x='thrust_factor', y='consumption', line_kws={"color": "C1"}, scatter_kws={'alpha':0.1})
ax.set_ylabel('Fuel consumption [l/trip]')
ax.set_xlabel(r'Aft thruster ratio: $\frac{C_{aft}}{C_{aft} +C_{fwd}}$')


ax.annotate('50\%-50\% \n aft-fwd thruster', (0.5,65), 
            xytext=(0.52,45), 
            arrowprops=dict(arrowstyle="->"), horizontalalignment='center', verticalalignment='top')

ax.annotate('90\%-10\% \n aft-fwd thruster', (0.90,32), 
            xytext=(0.9,60), 
            arrowprops=dict(arrowstyle="->"), horizontalalignment='center', verticalalignment='top')

ax.set_xlim(0.45,0.95)

glue('fig:thruster_correlation', fig,display=False)
fig.savefig('correlation.pdf')